# TP Elasticsearch


### **Alumno: Ivan Fabio Gutierrez Pinedo DNI 33437838**

Utilizando el archivo G20 provisto durante la clase (https://bit.ly/g20_Diplo2020), indexar los tweets del archivo, asegurándose de definir el mapping de forma correcta para los campos de interés. Notar que el mapping correcto no sólo depende del tipo de datos, sino de las consultas que deben hacerse sobre los mismos. De ser necesario o conveniente, se podría elegir indexar un campo de más de una forma.
Los campos de interés son:
*  “text”, “created_at”, “id”, “user.location”, “user.followers_count”, “place.bounding_box”,  “source”, “entities.hashtags”, “timestamp_ms”, “retweeted”
Las consultas de interés son:
1. Poder buscar teniendo en cuenta el patrón de mayúsculas o minúsculas, por ejemplo: “President” o “PRESIDENT” en el campo ‘text’, solo debe devolver documentos que tengan la palabra escrita con el mismo patrón de mayúsculas y minúsculas.
2. Poder buscar por palabras que compartan una misma raíz, por ejemplo: “pray”, haría un match con tweets que contengan: “prays”, “prayer”, “praying”, “prayers” en el campo ‘text’.
3. Consulta para poder buscar una ubicación en el campo ‘user.location’ y que el matching sea exacto, por ejemplo para las consultas “York” y “California”, no debería devolver documentos en donde user.location sea “New York” o “LA, California” por ejemplo.
4. Consulta para poder buscar tweets cuyos usuarios hayan abierto sus cuentas en un rango de fechas.
5. Consulta para poder buscar tweets que hayan sido posteados en un área específica,por ejemplo, en el área de “Washington DC” usando el campo “place.bounding_box”.

Investigacion para realizar el TP 
* https://www.youtube.com/watch?v=ma3BC8aPBfE&ab_channel=AmineM.Boulouma

In [1]:
from elasticsearch import Elasticsearch, helpers
import os, uuid
import json
from datetime import datetime
import dateutil.parser

In [2]:
# Configuration

mapping = {
  "properties": {
    "text": {
      "type": "text",
      "analyzer": "my_analyzer",
      "fields": {
        "case_sensitive": {
          "type": "text",
          "analyzer": "case_sensitive"
        }        
      }      
    },
    "created_at": {
      "type": "date",
      "format": "E MMM dd HH:mm:ss Z yyyy"
    },
    "user": {
      "properties": {
        "location": { "type": "keyword" }
      }
    },
    "place": {
      "properties": {
        "bounding_box": {
          "type": "geo_shape",
          "coerce": "true",
          }
      }
    }  
  }
}

setting = {
  "settings": {
    "analysis": {
      "analyzer": {
        "my_analyzer": {
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "my_stemmer"
          ]
        },
        "stem_analysis": {
          "tokenizer": "whitespace",
          "filter": [ "stemmer" ]
        },
      "case_sensitive" : {
             "type" : "custom",
             "tokenizer":    "standard",
             "filter": ["stop", "porter_stem" ]                    
         }
      },
      "filter": {
        "my_stemmer": {
          "type": "stemmer",
          "language": "english"#"light_english"#"english"
        }
      }
    },
    "index.mapping.total_fields.limit": 2000,
  }
}

In [3]:
mapping

{'properties': {'text': {'type': 'text',
   'analyzer': 'my_analyzer',
   'fields': {'case_sensitive': {'type': 'text',
     'analyzer': 'case_sensitive'}}},
  'created_at': {'type': 'date', 'format': 'E MMM dd HH:mm:ss Z yyyy'},
  'user': {'properties': {'location': {'type': 'keyword'}}},
  'place': {'properties': {'bounding_box': {'type': 'geo_shape',
     'coerce': 'true'}}}}}

In [4]:
index_name = 'g20_index'
es = Elasticsearch(HOST="http://localhost", PORT=9200)
exist_index = es.indices.exists(index_name)
if exist_index :
    es.indices.delete(index_name)
es.indices.create(index_name, body=setting)



{'acknowledged': True, 'shards_acknowledged': True, 'index': 'g20_index'}

In [5]:
es.indices.put_mapping(body=mapping, index=index_name)

{'acknowledged': True}

In [6]:
def get_data_from_file(file_name):
    if "/" in file_name or chr(92) in file_name:
        file = open(file_name, encoding="utf8", errors='ignore')
    else:
        # use the script_path() function to get path if none is passed
        file = open('./' + str(file_name), encoding="utf8", errors='ignore')
    data = [line.strip() for line in file]
    file.close()
    return data

def bulk_g20_ndjson_data(json_file, _index):
    json_list = get_data_from_file(json_file)
    index = None
    dict_index = None
    for doc in json_list:
        if '{"index"' in doc:
            dict_index = json.loads(doc)
        if '{"index"' not in doc and index != None:        
            yield {
                "_index": _index,
                #"_type": doc_type,
                "_id": dict_index.get('index').get('_id'),
                "_source": doc
            }
            dict_index = None
            
def bulk_g20_from_dict_data(g20_dict, _index):      
    for key, doc in g20_dict.items():
          
        yield {
            "_index": _index,
            #"_type": doc_type,
            "_id": key,
            "_source": doc
        }
        
def convert_g20_to_dict(file_name):    
    data = get_data_from_file('g20.json')
    index_list = []
    doc_list = []
    for row in data:
        if '{"index"' in row:
            dict_index = json.loads(row)
            index_list.append(dict_index)
        else :
            doc_list.append(row)


    g20_dict = { index_list[i].get('index').get('_id') : doc_list[i] for i in range(0, len(index_list)) } 
    return g20_dict


In [7]:
data = convert_g20_to_dict('g20.json')
helpers.bulk(es, bulk_g20_from_dict_data(data, index_name))

(9567, [])

---
1. Poder buscar teniendo en cuenta el patrón de mayúsculas o minúsculas, por ejemplo: “President” o “PRESIDENT” en el campo ‘text’, solo debe devolver documentos que tengan la palabra escrita con el mismo patrón de mayúsculas y minúsculas.


In [8]:
word = "PRESIDENT"
search_body = {
  "query": {
    "match": { "text.case_sensitive": word } 
  } 
  , "_source": ["text"]

}
es.search(body=search_body, index=index_name)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 9.121584,
  'hits': [{'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059864699968372737',
    '_score': 9.121584,
    '_source': {'text': '@realDonaldTrump Great Job Mr. PRESIDENT! Finally.. https://t.co/QePjSzlIi8'}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059868987855724544',
    '_score': 8.288621,
    '_source': {'text': '@WhiteHouse @realDonaldTrump #HELLO..MY PRESIDENT PRAY FOR..EVER..CONQUER..SMILE..SECOND..45.'}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059864875084775424',
    '_score': 7.193758,
    '_source': {'text': '@realDonaldTrump @MorriseyWV @CarolMillerWV TRAITOR TRUMP SIDED WITH RUSSIAN PRESIDENT PUTIN INSTEAD OF BELIEVING A… https://t.co/2Sn7oDOluZ'}}]}}

In [9]:
word = "President"
search_body = {
  "query": {
    "match": { "text.case_sensitive": word } 
  } 
  , "_source": ["text"]

}
es.search(body=search_body, index=index_name)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 509, 'relation': 'eq'},
  'max_score': 3.8845444,
  'hits': [{'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059868781495894021',
    '_score': 3.8845444,
    '_source': {'text': '@realDonaldTrump Come on, Mr President!!!!'}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059868897493557248',
    '_score': 3.8845444,
    '_source': {'text': '@realDonaldTrump Weak Baby President.'}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059868532975026178',
    '_score': 3.7342854,
    '_source': {'text': '@realDonaldTrump Thank you President Klump!'}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059868985058041856',
    '_score': 3.7342854,
    '_source': {'text': '@realDonaldTrump already done mr. President'}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059866586889273352',
    '_score': 3

---
2. Poder buscar por palabras que compartan una misma raíz, por ejemplo: “pray”, haría un match con tweets que contengan: “prays”, “prayer”, “praying”, “prayers” en el campo ‘text’.


In [10]:
#Testing custom analyzer
analisis = {
  "analyzer" : "my_analyzer",
  "text" : "pray prayed praying prayer prayers"
}
es.indices.analyze(body=analisis, index=index_name)

{'tokens': [{'token': 'prai',
   'start_offset': 0,
   'end_offset': 4,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'prai',
   'start_offset': 5,
   'end_offset': 11,
   'type': '<ALPHANUM>',
   'position': 1},
  {'token': 'prai',
   'start_offset': 12,
   'end_offset': 19,
   'type': '<ALPHANUM>',
   'position': 2},
  {'token': 'prayer',
   'start_offset': 20,
   'end_offset': 26,
   'type': '<ALPHANUM>',
   'position': 3},
  {'token': 'prayer',
   'start_offset': 27,
   'end_offset': 34,
   'type': '<ALPHANUM>',
   'position': 4}]}

In [11]:
word = "praying"
search_body = {
  "query": {
    "match": { "text": word } 
  } 
  , "_source": ["text"]

}
es.search(body=search_body, index=index_name)

{'took': 21,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10, 'relation': 'eq'},
  'max_score': 7.6182895,
  'hits': [{'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059868987855724544',
    '_score': 7.6182895,
    '_source': {'text': '@WhiteHouse @realDonaldTrump #HELLO..MY PRESIDENT PRAY FOR..EVER..CONQUER..SMILE..SECOND..45.'}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059864553037684736',
    '_score': 7.4205594,
    '_source': {'text': '@WhiteHouse @realDonaldTrump The American People are praying for you Mr President 🙏🇺🇸'}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059862775965446144',
    '_score': 7.4101014,
    '_source': {'text': '@POTUS   🙏🏻🙏🏻Pray For President Trump🙏🏻🙏🏻🙏🏻\n🙏🏻🙏🏻🙏🏻🙏🏻Pray For America🙏🏻🙏🏻🙏🏻🙏🏻🙏🏻\nIn the Mighty Name of Jesus Release… https://t.co/HY2yIhB7LR'}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059867694256795655'

---
3. Consulta para poder buscar una ubicación en el campo ‘user.location’ y que el matching sea exacto, por ejemplo para las consultas “York” y “California”, no debería devolver documentos en donde user.location sea “New York” o “LA, California” por ejemplo.


In [12]:
location_string = 'California'
search_body = {
  "query": {
    "match": {
      "user.location": location_string
    }
  }
  , "_source": ["id", "user.location"]
}

es.search(body=search_body, index=index_name)

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4, 'relation': 'eq'},
  'max_score': 6.9822435,
  'hits': [{'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059863755603947520',
    '_score': 6.9822435,
    '_source': {'id': 1059863755603947520,
     'user': {'location': 'California'}}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059867162557394944',
    '_score': 6.9822435,
    '_source': {'id': 1059867162557394944,
     'user': {'location': 'California'}}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059867742466039808',
    '_score': 6.9822435,
    '_source': {'id': 1059867742466039808,
     'user': {'location': 'California'}}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059868585873395713',
    '_score': 6.9822435,
    '_source': {'id': 1059868585873395713,
     'user': {'location': 'California'}}}]}}

---
4. Consulta para poder buscar tweets cuyos usuarios hayan abierto sus cuentas en un rango de fechas.


In [13]:
date_format ="%a %b %d %H:%M:%S %z %Y"

date_to = dateutil.parser.parse("2018-11-05T08:27:18-0000")
date_from = dateutil.parser.parse("2018-11-07T08:27:18-0000")

date_to_str = date_to.strftime(date_format)
date_from_str = date_from.strftime(date_format)

print(date_to_str)
print(date_from_str)

Mon Nov 05 08:27:18 +0000 2018
Wed Nov 07 08:27:18 +0000 2018


In [14]:
search_body = {
  "query": {
    "bool": {
      "must": [
        {
          "range": {
            "created_at": {
              "gte": date_to_str,
              "lte": date_from_str
            }
          }
        }
      ]
    }
  }
  , "_source": ["created_at"]
}

es.search(body=search_body, index=index_name)

{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 7856, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059862694315069440',
    '_score': 1.0,
    '_source': {'created_at': 'Tue Nov 06 17:38:52 +0000 2018'}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059862694931693571',
    '_score': 1.0,
    '_source': {'created_at': 'Tue Nov 06 17:38:52 +0000 2018'}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059862696294780929',
    '_score': 1.0,
    '_source': {'created_at': 'Tue Nov 06 17:38:52 +0000 2018'}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059862697016246273',
    '_score': 1.0,
    '_source': {'created_at': 'Tue Nov 06 17:38:52 +0000 2018'}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059862697314000896',
    '_score': 1.0,
    '_source': {'created_at': 'Tue Nov 06 17:3

---
5. Consulta para poder buscar tweets que hayan sido posteados en un área específica,por ejemplo, en el área de “Washington DC” usando el campo “place.bounding_box”.

In [15]:
washingtonDC_envelope = [[-77.72727881333427, 39.25855829997086],[-75.50806630658498, 37.8529507389647]]

search_body = { "query":{
    "bool": {
      "must": {
        "match_all": {}
      },
      "filter": {
        "geo_shape": {
          "place.bounding_box": {
            "shape": {
              "type": "envelope",
              "coordinates" : washingtonDC_envelope
            },
            "relation": "within"
                      }       
                    }
              }
            } 
    
    },
    "_source": ["created_at", "place.bounding_box"]
}

es.search(body=search_body, index=index_name)


{'took': 4,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059863726218719232',
    '_score': 1.0,
    '_source': {'created_at': 'Tue Nov 06 17:42:58 +0000 2018',
     'place': {'bounding_box': {'coordinates': [[[-77.252801, 39.102707],
         [-77.252801, 39.169487],
         [-77.163064, 39.169487],
         [-77.163064, 39.102707]]],
       'type': 'Polygon'}}}},
   {'_index': 'g20_index',
    '_type': '_doc',
    '_id': '1059867328819675136',
    '_score': 1.0,
    '_source': {'created_at': 'Tue Nov 06 17:57:17 +0000 2018',
     'place': {'bounding_box': {'coordinates': [[[-77.262146, 38.811091],
         [-77.262146, 38.865962],
         [-77.157684, 38.865962],
         [-77.157684, 38.811091]]],
       'type': 'Polygon'}}}}]}}